<a href="https://colab.research.google.com/github/oraziotorre/TTinsight/blob/main/TTinsight_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TTinsight


---


## Data Cleaning & Feature Engineering

In questo notebook vedremo tutto il processo per il caricamento, l'analisi e la pulizia dei dati. Sceglieremo poi le feature migliori e generaremo il dataset finale

Vogliamo addrestrare un modello che predica la probabilità di "W" nella colonna matcht result

# **Inizializzazione**

In [17]:
# Pandas per la gestione dei dati
import pandas as pd

# Gestione per One Hot Encoding
from sklearn.preprocessing import OneHotEncoder

# Per visualizzare i DataFrame su Google Colab in formato tabellare
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [18]:
dataset = pd.read_csv("raw_dataset.csv")

dataset

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ..."
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ..."
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ..."
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-1,"[1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6, ...","[0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8, 9, ..."
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-2,"[1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, ...","[0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 7, 7, 8, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72092,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-0,"[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3]","[0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10]"
72093,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-1,"[0, 1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 7, 7, 8, 8, ...","[1, 1, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7, ..."
72094,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-1,"[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ..."
72095,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ..."


# **Etichettatura**

Il dataset avrà una nuova colonna ```set_result``` che conterrà il risultato del set per ogni partita:

- **1** per la classe **Vincitori** del set
- **0** per la classe **Sconfitti** del set

Questo tipo di etichettatura è utile per i nostri LSTM e Regressione Logistica, dove l'obiettivo sarà quello di prevedere se un giocatore vincerà o perderà un set sulla base di altre informazioni.


In [19]:
# Funzione per convertire i punteggi da stringa a lista di interi
def convert_to_int_list(points_str):
    # Verifica che la stringa sia valida
    if isinstance(points_str, str):
        # Rimuove le parentesi quadre e suddivide la stringa per ottenere i numeri
        return list(map(int, points_str.strip('[]').split(', ')))
    else:
        return None


# Funzione per determinare il risultato del set
def calculate_set_result(player_points, opponent_points):
    if player_points:
        if player_points[-1] > opponent_points[-1]:
            return 1  # Vittoria per il giocatore
        else:
            return 0  # Sconfitta per il giocatore
    else:
        return -1  # Dati non conformi

# Inizializzo una lista vuota per la nuova colonna con le etichette
set_results = []

# Trasformo i punteggi in liste di interi utilizzando la funzione definita
dataset['player_points'] = dataset['player_points'].apply(convert_to_int_list)
dataset['opponent_points'] = dataset['opponent_points'].apply(convert_to_int_list)

# Itero su ogni riga e dai punteggi definisce l'etichetta
for index, row in dataset.iterrows():
    result = calculate_set_result(row['player_points'], row['opponent_points'])
    set_results.append(result)

# Aggiungo la nuova colonna 'set_result' al DataFrame
dataset['set_result'] = set_results

# Visualizzo il risultato
dataset




,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-1,"[1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6, ...","[0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-2,"[1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, ...","[0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 7, 7, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72092,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-0,"[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3]","[0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10]",0
72093,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-1,"[0, 1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 7, 7, 8, 8, ...","[1, 1, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7, ...",1
72094,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-1,"[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",0
72095,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1


# **Analisi dei dati**


## Controllo del bilanciamento dei dati
Una delle prime cose che possiamo verificare è se il dataset sia **bilanciato**. Per cui, controlliamo il numero di istanze **1** e **0** della colonna *set_result*

In [20]:
# Numero di elementi per la classe "Vincitori"
print(len(dataset[(dataset['set_result'] == 1)]))
# Numero di elementi per la classe "Sconfitti"
print(len(dataset[(dataset['set_result'] == 0)]))

# Totale
print(len(dataset))


35234
34373
72097


Notiamo che abbiamo un dataset non troppo sbilanciato

## **Analisi delle caratteristiche e delle distribuzioni dei dati**
Usiamo `describe()` per avere una overview sulle distribuzioni dei dati numerici

In [21]:
# Verifico la correttezza dei campi numerici
dataset.describe()

,event_id,stage_id,match_duration,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,sets_required_to_win,set_result
count,72097.000000,72097.000000,72097.000000,72097.000000,17558.000000,72097.000000,17558.000000,72097.000000,72097.000000,72097.000000,72097.000000
mean,2702.487149,1556.146095,2058.918221,123932.242964,125566.084919,123741.393595,126035.559745,2.107397,2.066202,3.146705,0.454166
std,156.934388,5453.980041,1575.258589,17247.564085,17316.489865,17089.894592,18276.388290,1.245484,1.255210,0.353971,0.563007
min,2234.000000,100.000000,0.000000,100001.000000,100001.000000,90051.000000,100032.000000,0.000000,0.000000,2.000000,-1.000000
25%,2569.000000,200.000000,1426.000000,114716.000000,115754.000000,114715.000000,115548.000000,1.000000,1.000000,3.000000,0.000000
50%,2701.000000,600.000000,1914.000000,121408.000000,122421.000000,121103.000000,122428.000000,3.000000,3.000000,3.000000,0.000000
75%,2869.000000,1200.000000,2430.000000,132132.000000,133124.000000,132022.000000,133124.000000,3.000000,3.000000,3.000000,1.000000
max,2997.000000,110005.000000,77074.000000,212088.000000,212090.000000,211762.000000,211733.000000,5.000000,4.000000,5.000000,1.000000


Notiamo che nella colonna "set_required_to_win" sono presenti dei valori rari non nel nostro interesse (2 e 5)

## **Controllo dei valori nulli**

Verifico i valori mancanti nei dati

In [22]:
# Identificare i valori NaN nel dataset
nan_mask = dataset.isna()

# Conta i valori NaN (valori mancanti) per ciascuna colonna del dataset
nan_count = nan_mask.sum()

# Verifico i valori null del raw_dataset
nan_count


,0
event_id,0
match_id,0
match_format,0
players_gender,0
match_stage,0
stage_id,0
match_duration,0
match_start_time,0
player_id,0
player_2_id,54539


#**Operazione sulle feature**


Facciamo alcune considerazioni sulle caratteristiche del nostro dataset e identifichiamo possibili **miglioramenti** per ottimizzarlo in vista del prossimo training del modello:

* Possibile **trasformare i formati T** nella colonna ```match_format``` in formati che siano o S o D

* La colonna ```match_stage``` può essere modificata per attribuire **maggiore importanza alle partite finali** (FNL) rispetto a tutte le altre, migliorando il peso di queste informazioni nel training del modello.

* Le colonne ```sets_required_to_win``` e ```current_match_state``` fanno entrambe riferimento al numero di set necessari per vincere una partita. Per rendere il dataset più chiaro e organizzato, possiamo **sostituirle** con due nuove colonne: ```sets_to_win``` e ```opponent_sets_to_win```, che rappresentano rispettivamente il numero di set necessari al giocatore e all'avversario per vincere la partita.

* Per la colonna ```players_gender``` si può semplificare il formato tramite l'operazione di ***One-Hot Encoding*** nel caso di dataset_all_matches o tramite una semplice codifica binaria nel caso del dataset_single_matches

*  Le colonne ```match_format```, ```player_id```, ```player_2_id``` **non sono necessarie** per il training del modello e possono essere cancellate




## **Colonna *match_format***

Modifica la colonna *match_format*, sostituendo tutti i valori ' T ' (partita di team) con ' S ' (partita di singolo) o ' D ' (partita di doppio)

Viene eseguita questa operazione per **evitare la rimozione di valori** che potrebbero ritornarci utile successivamente nello sviluppo del nostro modello

In [23]:
# Funzione per determinare il match_format quando è Team (Singolo o Doppio)
def determine_match_format(player_2_id):
    # Controlla se il valore di player_2_id è NaN (valore mancante)
    if pd.isna(player_2_id):
        return 'S'  # Ritorna 'S' per indicare una partita singola
    else:
        return 'D'  # Ritorna 'D' per indicare una partita doppia

# Applica la funzione determine_match_format alla colonna 'player_2_id' del dataset
# Modificando il valore nella colonna match_format di tipo S o D
dataset['match_format'] = dataset['player_2_id'].apply(determine_match_format)

# Mostra il dataset aggiornato
dataset


,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-1,"[1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6, ...","[0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-2,"[1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, ...","[0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 7, 7, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72092,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-0,"[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3]","[0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10]",0
72093,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-1,"[0, 1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 7, 7, 8, 8, ...","[1, 1, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7, ...",1
72094,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-1,"[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",0
72095,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1


In base a ciò che abbiamo appreso precedentemente da `describe()`, rimuoviamo i valori non desiderati della colonna *sets_required_to_win* uguali a 2 o 5.

Questi valori rari potrebbero rappresentare **formati di gioco non standard** (es. partite amichevoli o partite abbreviate per motivi eccezionali), non rilevanti per l'analisi



In [24]:
# Rimuoviamo le righe contenenti set con punteggi rari che non vogliamo considerare
dataset = dataset[~dataset['sets_required_to_win'].isin([2, 5])]

dataset.describe()

,event_id,stage_id,match_duration,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,sets_required_to_win,set_result
count,72093.000000,72093.000000,72093.000000,72093.000000,17558.000000,72093.000000,17558.000000,72093.000000,72093.000000,72093.000000,72093.000000
mean,2702.497746,1556.146436,2058.904013,123932.527971,125566.084919,123741.600433,126035.559745,2.107375,2.066206,3.146686,0.454205
std,156.932278,5454.129806,1575.287652,17248.000098,17316.489865,17090.133972,18276.388290,1.245375,1.255245,0.353795,0.562989
min,2234.000000,100.000000,0.000000,100001.000000,100001.000000,90051.000000,100032.000000,0.000000,0.000000,3.000000,-1.000000
25%,2569.000000,200.000000,1426.000000,114716.000000,115754.000000,114715.000000,115548.000000,1.000000,1.000000,3.000000,0.000000
50%,2701.000000,600.000000,1914.000000,121408.000000,122421.000000,121103.000000,122428.000000,3.000000,3.000000,3.000000,0.000000
75%,2869.000000,1200.000000,2430.000000,132132.000000,133124.000000,132022.000000,133124.000000,3.000000,3.000000,3.000000,1.000000
max,2997.000000,110005.000000,77074.000000,212088.000000,212090.000000,211762.000000,211733.000000,4.000000,4.000000,4.000000,1.000000


Eliminiamo i valori nulli della colonna *player_points*

In [25]:
# Rimuoviamo tutte le righe del dataset in cui la colonna 'player_points' contiene valori NaN
dataset = dataset.dropna(subset=["player_points"])

# Verifichiamo che non esistano più righe con valori NaN nella colonna 'player_points'
dataset[dataset['player_points'].isnull()]

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result


##**Colonna *match_stage***

Al posto della colonna *match_stage* inseriamo *is_final* in modo da semplificarne i valori in:
- **FNL = 1**
- **Tutti gli altri stage del torneo = 0**

In questo modo possiamo dare maggiore importanza alle partite finali (FNL) rispetto a tutte le altre, **migliorando il peso di queste informazioni** nel training del modello.

In [26]:
# Creiamo una nuova colonna 'is_final' che vale 1 se 'match_stage' è 'FNL' (finale), altrimenti 0
dataset['is_final'] = (dataset['match_stage'] == 'FNL').astype(int)

# Rimuoviamo la colonna 'match_stage' dal dataset poiché non è più necessaria
dataset = dataset.drop(columns=['match_stage'])

# Troviamo la posizione della colonna 'players_gender' nel dataset
players_gender_index = dataset.columns.get_loc('players_gender')

# Estraiamo la colonna 'is_final' e la inseriamo subito dopo 'players_gender'
is_final_column = dataset.pop('is_final')
dataset.insert(players_gender_index + 1, 'is_final', is_final_column)

# Mostra il dataset aggiornato
dataset


<ipython-input-26-a800c8080596>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['is_final'] = (dataset['match_stage'] == 'FNL').astype(int)


,event_id,match_id,match_format,players_gender,is_final,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-1,"[1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6, ...","[0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-2,"[1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, ...","[0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 7, 7, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72092,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-0,"[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3]","[0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10]",0
72093,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-1,"[0, 1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 7, 7, 8, 8, ...","[1, 1, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7, ...",1
72094,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-1,"[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",0
72095,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1


##**Colonne *sets_required_to_win* e *current_match_state***

Il codice calcola il **numero di set mancanti alla vittoria** per un giocatore e per il suo avversario, basandosi su informazioni presenti nel dataset e creando due nuove colonne ```final_set``` e ```opponent_final_set```

Determiniamo se un giocatore o il suo avversario è a un set dalla vittoria
- Se il giocatore è un set dalla vittoria poniamo **final_set pari a 1**
- Se l'avversario è un set dalla vittoria poniamo **opponent_final_set pari a 1**
- Altrimenti sarà **pari a 0**

Elimina le colonne originali ```sets_required_to_win``` e ```current_match_state```, ormai non più necessarie.

Le colonne originali contenevano informazioni in un formato più complesso:
- *current_match_state* era una stringa del tipo **"x-y"**, che richiede parsing per separare i valori x e y.
- *sets_required_to_win* da sola **non fornisce** un'indicazione diretta del progresso del match.
Le nuove colonne forniscono le stesse informazioni in modo più esplicito, eliminando la necessità di operazioni intermedie (come lo split delle stringhe).

In [27]:
# Calcola la differenza tra i set richiesti per vincere e i set vinti dal giocatore e dall'avversario
final_set_diff = dataset['sets_required_to_win'] - dataset['current_match_state'].str.split('-', expand=True)[0].astype(int)
opponent_final_set_diff = dataset['sets_required_to_win'] - dataset['current_match_state'].str.split('-', expand=True)[1].astype(int)

# Crea una colonna 'final_set' che vale 1 se il giocatore è a un set dalla vittoria, altrimenti 0
dataset['final_set'] = (final_set_diff == 1).astype(int)

# Crea una colonna 'opponent_final_set' che vale 1 se l'avversario è a un set dalla vittoria, altrimenti 0
dataset['opponent_final_set'] = (opponent_final_set_diff == 1).astype(int)

# La posizione della colonna is_final viene determinata con get_loc.
is_final_index = dataset.columns.get_loc('is_final')

# Le nuove colonne final_set e opponent_final_set vengono temporaneamente rimosse (pop) e poi reinserite subito dopo la colonna is_final usando insert.
final_set_column = dataset.pop('final_set')
opponent_final_set_column = dataset.pop('opponent_final_set')

# Riposiziona le colonne 'final_set' e 'opponent_final_set' subito dopo 'is_final'
dataset.insert(is_final_index + 1, 'final_set', final_set_column)
dataset.insert(is_final_index + 2, 'opponent_final_set', opponent_final_set_column)

# Elimina le colonne originali 'sets_required_to_win' e 'current_match_state'
dataset.drop(['sets_required_to_win', 'current_match_state'], axis=1, inplace=True)

# Mostra il dataset aggiornato
dataset


,event_id,match_id,match_format,players_gender,is_final,final_set,opponent_final_set,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,1,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6, ...","[0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,1,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, ...","[0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 7, 7, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72092,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3]","[0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10]",0
72093,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[0, 1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 7, 7, 8, 8, ...","[1, 1, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7, ...",1
72094,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",0
72095,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1


##**Colonna *player_points***

Ogni progressione dei punteggi viene convertita in un vettore binario, dove ogni elemento rappresenta un punto nel set. Il valore del vettore sarà 1 se il giocatore ha fatto un punto in quel particolare istante del set, e 0 altrimenti.

In [28]:
# Converte i numeri in un altro formato
def transform_player_points(points):
    transformed = []

    for i in range(len(points)):
        points[i] = int(points[i])  # Conversione del punteggio in int per evitare problemi di tipo

        # Per il primo punteggio, aggiungiamo 1 se il punteggio è maggiore di 0, altrimenti 0
        if i == 0:
            transformed.append(1 if points[i] > 0 else 0)
        else:
            # Aggiungiamo 1 se il punteggio è maggiore del precedente, altrimenti 0
            transformed.append(1 if points[i] > points[i - 1] else 0)

    return transformed

dataset['player_points'] = dataset['player_points'].apply(transform_player_points)

dataset


,event_id,match_id,match_format,players_gender,is_final,final_set,opponent_final_set,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,1,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, ...","[0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,1,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, ...","[0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 7, 7, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72092,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]","[0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10]",0
72093,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, ...","[1, 1, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7, ...",1
72094,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, ...","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",0
72095,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1


Per ogni istanza di un set con punteggio finale (ad esempio, 10-5), la funzione crea tutte le possibili situazioni intermedie dei punteggi. Questo significa che, dato il punteggio finale del set, vengono estratte tutte le progressioni dei punteggi intermedi (ad esempio, 1-0, 2-0, 3-1, e così via) fino al punteggio finale. In pratica, la funzione analizza la sequenza di punti per ricostruire ogni stato intermedio del set, consentendo una visione più dettagliata dell'andamento della partita e della sua evoluzione.

In [29]:
import ast

def generate_score_sequence(player_points):

    # Se player_points è una stringa che sembra una lista, la convertiamo in una lista effettiva
    if isinstance(player_points, str):
      player_points = ast.literal_eval(player_points)

    # Verifica che player_points sia effettivamente una lista di numeri
    if isinstance(player_points, list):
        player_points = [int(x) for x in player_points]

    # Crea una sequenza delle liste
    sequence = [list(player_points[:i]) for i in range(1, len(player_points) + 1)]

    return sequence

# Generiamo ed esplodiamo la colonna
dataset['player_points'] = dataset['player_points'].apply(generate_score_sequence)
dataset = dataset.explode('player_points', ignore_index=True)

dataset

,event_id,match_id,match_format,players_gender,is_final,final_set,opponent_final_set,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",[0],"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207431,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207432,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207433,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207434,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


Rimuovo le sequenze di punteggi che non ci interessano:

- **Sequenze che arrivano ai vantaggi**: In questi casi, la previsione del vincitore diventa troppo complessa a causa delle molteplici combinazioni possibili, quindi queste sequenze vengono escluse.
- **Sequenze che arrivano al set point**: Quando un giocatore è a set point, senza vantaggi, il risultato è quasi certo, poiché chi ha 10 punti è ovviamente il vincitore, rendendo la previsione inutile.
- **Situazioni parziali del set con meno di 6 punti giocati**: Escludiamo queste sequenze in quanto non rappresentano uno scenario completo per una previsione accurata.
- **Situazioni parziali con una differenza maggiore di 3 punti**: Quando la differenza tra i punteggi è troppo alta, la probabilità che il giocatore in vantaggio vinca è elevata, rendendo la previsione poco interessante o informativa.

Questi filtri sono applicati anche per ridurre la dimensionalità del dataset, migliorando la qualità del modello.

In [30]:
def remove_rows_with_large_player_points(df):

    # Funzione che verifica i pounteggi
    def check_points(player_points):

      if isinstance(player_points, str):
          player_points = ast.literal_eval(player_points)

      # Conta il numero di 0 e 1
      count_zeros = player_points.count(0)
      count_ones = player_points.count(1)

      # Verifica se la somma di 0 e 1 è inferiore a 10
      if count_zeros+count_ones<20 and count_zeros < 10 and count_ones < 10 and count_zeros+count_ones>6 and abs(count_ones-count_zeros)<4:
          return True
      else:
          return False


    # Filtra le righe
    df_filtered = df[df['player_points'].apply(check_points)]
    return df_filtered

dataset = remove_rows_with_large_player_points(dataset)
dataset

,event_id,match_id,match_format,players_gender,is_final,final_set,opponent_final_set,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,player_points,opponent_points,set_result
6,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0, 1, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
7,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0, 1, 1, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
8,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0, 1, 1, 0, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
9,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0, 1, 1, 0, 0, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
10,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207428,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207430,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207431,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207432,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


Aggiungiamo nuove caratteristiche al dataset in base alla sequenza di punti (player_points) per ogni partita. Le nuove feature incluse sono:

- **Differenza di rimonta corrente (last_comeback)**: Rappresenta la differenza tra il numero di punti consecutivi vinti dal giocatore A e dal giocatore B, fornendo una misura dell'attuale slancio di gioco.
- **Lunghezza della sequenza (points_length)**: Indica il numero totale di punti giocati durante la sequenza, utile per analizzare l'intensità della partita.
- **Differenza di punteggio1 (diff)**: Calcola la differenza totale dei punti tra i due giocatori, utile per capire quale giocatore sta conducendo nel punteggio.
- **Differenza di punteggio2 (half_diff)**: Calcola la differenza totale dei punti tra i due giocatori nell'ultima metà della partita

- **Differenza di punteggio3 (third_diff)**: Calcola la differenza totale dei punti tra i due giocatori nell'ultimo terzo della partita

Queste feature sono aggiunte per poter addestare i modelli che non sono capaci di lavorare direttamente sulle sequenze di punteggi

In [34]:
def aggiungi_rimonte_per_punti(dataset):
    # Liste per memorizzare i risultati
    points_lengths = []
    last_comeback = []
    diff = []
    half_diff = []
    third_diff = []

    for points in dataset['player_points']:

        punti_a = punti_b = rimonta_a = rimonta_b = 0
        length = len(points)

        for point in points:
            if point == 1:
                rimonta_a += 1
                rimonta_b = 0
                punti_a += 1
            elif point == 0:
                rimonta_b += 1
                rimonta_a = 0
                punti_b += 1

        last_comeback.append(rimonta_a - rimonta_b)
        points_lengths.append(length)
        diff.append(punti_a - punti_b)

        punti_a = punti_b = 0
        start_index = length // 2

        for point in points[start_index:]:
            if point == 1:
                punti_a += 1
            elif point == 0:
                punti_b += 1

        half_diff.append(punti_a - punti_b)


        punti_a = punti_b = 0
        start_index = 2*(length // 3)+1

        for point in points[start_index:]:
            if point == 1:
                punti_a += 1
            elif point == 0:
                punti_b += 1

        third_diff.append(punti_a - punti_b)


    # Aggiunge le nuove colonne al dataset
    dataset = dataset.assign(
        comeback=last_comeback,
        length=points_lengths,
        diff=diff,
        half_diff=half_diff,
        third_diff=third_diff,
    )

    return dataset


dataset = aggiungi_rimonte_per_punti(dataset)
columns = dataset.columns.tolist()
columns.remove('set_result')
columns.append('set_result')
dataset = dataset[columns]
dataset

,event_id,match_id,match_format,players_gender,is_final,final_set,opponent_final_set,stage_id,match_duration,match_start_time,...,opponent_sets_won,match_scores,player_points,opponent_points,comeback,length,diff,half_diff,third_diff,set_result
6,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,...,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0, 1, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",2,7,1,2,2,0
7,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,...,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0, 1, 1, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",-1,8,0,0,1,0
8,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,...,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0, 1, 1, 0, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",-2,9,-1,-1,-2,0
9,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,...,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0, 1, 1, 0, 0, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",1,10,0,1,-1,0
10,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,...,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",2,11,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207428,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,...,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",-5,9,-3,-5,-2,0
1207430,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,...,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",1,11,-3,-4,-2,0
1207431,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,...,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",2,12,-2,-2,1,0
1207432,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,...,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",-1,13,-3,-3,0,0



##**Operazione su *single_matches_dataset***

Facciamo adesso alcune operazioni specifiche sulle feature su un dataset composto unicamente da partite di singolo

Filtrando solo i dati di singolo possiamo concentrarci su un **sottoinsieme specifico**, migliorando la qualità delle analisi.

In [ ]:
# Rimuoviamo dal dataset le partite di doppio
rows_to_remove = dataset[dataset['match_format'] == 'D'].index

# Rimuoviamo le righe corrispondenti dal dataset originale
dataset_singles = dataset.drop(index=rows_to_remove)

dataset_singles

,event_id,match_id,match_format,players_gender,is_final,final_set,opponent_final_set,stage_id,match_duration,match_start_time,...,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,player_points,opponent_points,set_result,last_comeback,points_diff,points_length
132,2234,TTEMTEAM--------------FNL-00010001--------,S,M,1,0,0,10001,2387,2021-11-19 23:25:33,...,NaN,3,2,"11-8,11-7,7-11,7-11,11-5","[0, 1, 1, 0, 0, 1, 0]","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",1,-1,-1,7
133,2234,TTEMTEAM--------------FNL-00010001--------,S,M,1,0,0,10001,2387,2021-11-19 23:25:33,...,NaN,3,2,"11-8,11-7,7-11,7-11,11-5","[0, 1, 1, 0, 0, 1, 0, 1]","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",1,1,0,8
134,2234,TTEMTEAM--------------FNL-00010001--------,S,M,1,0,0,10001,2387,2021-11-19 23:25:33,...,NaN,3,2,"11-8,11-7,7-11,7-11,11-5","[0, 1, 1, 0, 0, 1, 0, 1, 0]","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",1,-1,-1,9
135,2234,TTEMTEAM--------------FNL-00010001--------,S,M,1,0,0,10001,2387,2021-11-19 23:25:33,...,NaN,3,2,"11-8,11-7,7-11,7-11,11-5","[0, 1, 1, 0, 0, 1, 0, 1, 0, 1]","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",1,1,0,10
136,2234,TTEMTEAM--------------FNL-00010001--------,S,M,1,0,0,10001,2387,2021-11-19 23:25:33,...,NaN,3,2,"11-8,11-7,7-11,7-11,11-5","[0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1]","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",1,2,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207428,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,...,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0,-5,-3,9
1207430,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,...,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0,1,-3,11
1207431,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,...,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0,2,-2,12
1207432,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,...,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0,-1,-3,13


## **Eliminazione delle colonne non adatte all'addestramento**

Vantaggi di questa operazione:
- **Eliminando colonne non necessarie**, il dataset diventa più semplice e focalizzato sui dati utili.
- Rimuovendo informazioni irrilevanti, l'addestramento risulta più chiaro e veloce, **evitando di appesantire** il modello con dati superflui.
- Con meno colonne da analizzare, è più **facile comprendere la struttura del dataset** e concentrarsi sugli aspetti importanti (ad esempio, il genere del giocatore o i set vinti).

In [ ]:
# Definiamo le colonne da eliminare
columns_to_drop = ['event_id', 'match_id', 'stage_id', 'match_duration',
                   'players_gender','match_start_time','opponent_id', 'opponent_2_id',
                  'player_sets_won', 'opponent_sets_won', 'match_scores',
                  'opponent_points','match_format', 'player_id', 'player_2_id']

# Rimuovo le colonne che non mi interessano
dataset = dataset.drop(columns=columns_to_drop, axis=1)
dataset_singles = dataset_singles.drop(columns=columns_to_drop, axis=1)

dataset_singles

,is_final,final_set,opponent_final_set,player_points,set_result,last_comeback,points_diff,points_length
132,1,0,0,"[0, 1, 1, 0, 0, 1, 0]",1,-1,-1,7
133,1,0,0,"[0, 1, 1, 0, 0, 1, 0, 1]",1,1,0,8
134,1,0,0,"[0, 1, 1, 0, 0, 1, 0, 1, 0]",1,-1,-1,9
135,1,0,0,"[0, 1, 1, 0, 0, 1, 0, 1, 0, 1]",1,1,0,10
136,1,0,0,"[0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1]",1,2,1,11
...,...,...,...,...,...,...,...,...
1207428,0,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0]",0,-5,-3,9
1207430,0,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]",0,1,-3,11
1207431,0,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]",0,2,-2,12
1207432,0,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]",0,-1,-3,13


In [ ]:
# Salviamo il dataset pulito dai valori indesiderati
dataset.to_csv("cleaned_dataset.csv", index=False)

In [ ]:
# Salviamo il dataset finale delle partite di singolo
dataset_singles.to_csv("singles_matches_dataset.csv", index=False)